### 제품명 정규화(brand명 제거까지)

In [1]:
import re
import pandas as pd

# =========================
# 1) 정규화 패턴
# =========================
PROMO_PATTERNS = [
    r"\[.*?\]",
    r"\(.*?\)",
    r"(?i)\bset\b",
    r"기획팩|기획세트|기획|한정기획|한정|증량기획|증량|리뉴얼|어워즈|올영픽|온라인몰\s*단독기획|단독기획|단품",
    r"더블\s*기획|트리플\s*기획|1\+1|2\+1|3\+1",
    r"증정|사은품|증정품|추가\s*증정|리필|리필팩|대용량|미니|샘플|사은|특가|한정판",
    r"\d+\s*종\s*(?:중\s*)?택\s*1",
    r"(?:\d+\s*종\s*)?중\s*택\s*1",
    r"\d+\s*입|[0-9]+\s*개입|[0-9]+\s*개",
    r"(?:단품|기획|세트|본품|리필|증정품|사은품)(?:\s*/\s*(?:단품|기획|세트|본품|리필|증정품|사은품))*\s*(?:중\s*)?택\s*1",
    r"\d+\s*중\s*택\s*1",
    r"\b\d+\b$",
    r"\d+\s*종\b",
    r"\b세트\b",
    r"\d+\s*중\s*택\s*(?:1)?",
    r"\b택\s*1\b",
    r"\+.*$",
]

VOLUME_PATTERNS = [
    r"\d+(?:\.\d+)?\s*(?:ml|mL|ML)\b",
    r"\d+(?:\.\d+)?\s*(?:g|G)\b",
    r"\d+(?:\.\d+)?\s*(?:oz|OZ)\b",
]

def normalize_product_name(raw_name: str) -> str:
    if pd.isna(raw_name):
        return ""
    s = str(raw_name).strip()

    for pat in PROMO_PATTERNS:
        s = re.sub(pat, " ", s)

    for pat in VOLUME_PATTERNS:
        s = re.sub(pat, " ", s)

    s = s.replace("·", " ").replace("/", " ").replace("|", " ")
    s = re.sub(r"[_\-–—]+", " ", s)
    s = re.sub(r"[^\w가-힣\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def remove_leading_brand_if_match(canonical_name: str, brand: str) -> str:
    if pd.isna(canonical_name):
        return ""
    s = str(canonical_name).strip()

    if pd.isna(brand) or str(brand).strip() == "":
        return s

    b = str(brand).strip()
    if not s:
        return s

    parts = s.split()
    first = parts[0]

    if first.lower() == b.lower():
        s = " ".join(parts[1:]).strip()
        return re.sub(r"\s+", " ", s).strip()

    if s.lower().startswith(b.lower() + " "):
        s = s[len(b):].strip()
        return re.sub(r"\s+", " ", s).strip()

    return s

def build_product_key(brand: str, canonical_name: str) -> str:
    b = "" if pd.isna(brand) else str(brand).strip()
    n = "" if pd.isna(canonical_name) else str(canonical_name).strip()
    return re.sub(r"\s+", " ", f"{b}__{n}".strip("_")).strip()

def pick_col(df: pd.DataFrame, candidates: list[str], required_name: str):
    for c in candidates:
        if c in df.columns:
            return c
    raise KeyError(f"필수 컬럼({required_name}) 없음. 후보={candidates} / 현재컬럼={list(df.columns)}")

# =========================
# 2) CSV 적용 (원본 열 100% 유지 + 새 열만 추가)
# =========================
in_path = "products_ingredients_final_essence.csv"
df = pd.read_csv(in_path)

brand_col = pick_col(df, ["brand", "브랜드"], "brand")
name_col  = pick_col(df, ["product_name", "productName", "원래제품명", "제품명"], "product_name")

# ✅ 원본 열은 그대로 두고, 새 열만 추가
df["canonical_name"] = df[name_col].apply(normalize_product_name)
df["canonical_name"] = df.apply(lambda r: remove_leading_brand_if_match(r["canonical_name"], r[brand_col]), axis=1)
df["product_key"] = df.apply(lambda r: build_product_key(r[brand_col], r["canonical_name"]), axis=1)

print("원본 컬럼 개수:", len(df.columns))
print(df[[name_col, "canonical_name", "product_key"]].head(20).to_string(index=False))

out_path = "products_ingredients_final_essence_brandremoved_keepcols.csv"
df.to_csv(out_path, index=False, encoding="utf-8-sig")
print(f"\n저장 완료: {out_path}")


원본 컬럼 개수: 11
                                                     product_name              canonical_name                        product_key
   [2025 어워즈] 아누아 피디알엔 히알루론산 캡슐 100 세럼 30mL (+리필30mL+크림10mL+패드2매)        피디알엔 히알루론산 캡슐 100 세럼          아누아__피디알엔 히알루론산 캡슐 100 세럼
                  [1월올영픽/진정탄력] 차앤박 더마앤서 액티브 부스트 PDRN 앰플 30ml 더블기획     더마앤서 액티브 부스트 PDRN 앰플 더블       차앤박__더마앤서 액티브 부스트 PDRN 앰플 더블
                     [단독/1+1] 메디힐 마데카소사이드 흔적 리페어 세럼 40+40ml 더블 기획        마데카소사이드 흔적 리페어 세럼 40          메디힐__마데카소사이드 흔적 리페어 세럼 40
[2025 어워즈/미백천재앰플] 메디큐브 PDRN 핑크앰플 30ml 어워즈 기획 (+리필50ml+세럼1.5ml*5매)                   PDRN 핑크앰플                    메디큐브__PDRN 핑크앰플
                          [1월올영픽/키링증정] 이니스프리 레티놀 시카 앰플 30ml 더블 기획                레티놀 시카 앰플 더블                이니스프리__레티놀 시카 앰플 더블
               [2025 어워즈 1위] 토리든 다이브인 저분자 히알루론산 세럼 100ml 어워즈 한정기획           다이브인 저분자 히알루론산 세럼             토리든__다이브인 저분자 히알루론산 세럼
             [1월올영픽/보습광채] 차앤박 프로폴리스 에너지 액티브 앰플 30ml 2입 기획 (+15ml)            프로폴리스 에

In [2]:
import numpy as np
print(np)
print(getattr(np, "__version__", "NO_VERSION"))


<module 'numpy' (<_frozen_importlib_external._NamespaceLoader object at 0x1113e0280>)>
NO_VERSION


### 중복되는 제품의 경우 첫번째 행만 남기고 제거

In [ ]:
import pandas as pd

# =========================
# 0) 입력/출력 경로
# =========================
in_path = "/mnt/data/table2_products_normalized_reordered.csv"
out_path = "/mnt/data/table2_products_normalized_reordered_dedup.csv"

df = pd.read_csv(in_path)

# =========================
# 1) 중복 확인 (canonical_name 기준)
# =========================
key_col = "canonical_name"

# 중복되는 행만 뽑기 (첫 행 포함)
dup_mask = df.duplicated(key_col, keep=False)
df_dups = df[dup_mask].copy()

print("=" * 80)
print(f"[중복 요약] 기준 컬럼: {key_col}")
print("=" * 80)

if df_dups.empty:
    print("중복이 없습니다. (canonical_name 기준)")
else:
    # 중복 그룹 크기 요약
    dup_counts = (
        df_dups.groupby(key_col)
        .size()
        .sort_values(ascending=False)
    )

    print(f"중복 그룹 수: {dup_counts.shape[0]}")
    print(f"중복 행 수(전체): {df_dups.shape[0]}")
    print("\n중복 상위 20개 (canonical_name -> count):")
    print(dup_counts.head(20).to_string())

    # 어떤 행이 중복인지 상세 출력 (상위 N개 그룹만)
    TOP_N_GROUPS = 10
    top_names = dup_counts.head(TOP_N_GROUPS).index.tolist()

    # 출력 컬럼(있는 것만 출력)
    show_cols_candidates = [
        "product_id", "category", "brand",
        "canonical_name", "ingredients", "product_name", "product_key"
    ]
    show_cols = [c for c in show_cols_candidates if c in df.columns]

    print("\n" + "=" * 80)
    print(f"[중복 상세] 상위 {TOP_N_GROUPS}개 canonical_name 그룹의 행들")
    print("=" * 80)

    df_dups_top = df_dups[df_dups[key_col].isin(top_names)].copy()
    df_dups_top = df_dups_top.sort_values([key_col, "product_id"], kind="stable")
    print(df_dups_top[show_cols].to_string(index=False))

# =========================
# 2) 중복 제거 (첫 번째 행만 남김)
# =========================
df_dedup = df.drop_duplicates(subset=[key_col], keep="first").copy()

print("\n" + "=" * 80)
print("[Dedup 결과]")
print("=" * 80)
print(f"원본 행 수: {df.shape[0]}")
print(f"Dedup 후 행 수: {df_dedup.shape[0]}")
print(f"삭제된 행 수: {df.shape[0] - df_dedup.shape[0]}")

# =========================
# 3) 저장
# =========================
df_dedup.to_csv(out_path, index=False, encoding="utf-8-sig")
print(f"\n저장 완료: {out_path}")


In [ ]:
import pandas as pd

# =========================
# 0) 입력 경로
# =========================
in_path = "/Users/eunjin/Downloads/table2_products_normalized_reordered.csv"
df = pd.read_csv(in_path)

key_col = "canonical_name"
id_col = "product_id"

# =========================
# 1) 삭제될 행들만 추출
#    keep="first" → 첫 행을 제외한 나머지가 삭제 대상
# =========================
deleted_mask = df.duplicated(subset=[key_col], keep="first")
df_deleted = df[deleted_mask].copy()

print("=" * 80)
print("[삭제 대상 행 요약]")
print("=" * 80)
print(f"삭제될 행 수: {df_deleted.shape[0]}")

# =========================
# 2) 삭제된 product_id 출력
# =========================
deleted_ids = df_deleted[id_col].tolist()

print("\n[삭제된 product_id 목록]")
print("-" * 80)
for pid in deleted_ids:
    print(pid)

# =========================
# 3) 필요하면 파일로 저장
# =========================
out_ids_path = "/Users/eunjin/Downloads/deleted_product_ids.csv"
pd.DataFrame({id_col: deleted_ids}).to_csv(out_ids_path, index=False, encoding="utf-8-sig")

print(f"\nproduct_id 목록 저장 완료: {out_ids_path}")


[삭제 대상 행 요약]
삭제될 행 수: 60

[삭제된 product_id 목록]
--------------------------------------------------------------------------------
S15
S29
S31
S32
S39
S49
S50
S63
S97
S98
S99
S104
S109
S121
S125
S135
S141
S153
S163
S167
S170
S172
S173
S175
S176
S177
S188
S189
S196
S204
S218
S226
S230
S236
S248
S252
S258
S263
S265
S267
S269
S271
S273
S279
S288
S294
S295
S296
S301
S302
S306
S311
S314
S316
S319
S322
S324
S327
S328
S329

product_id 목록 저장 완료: /Users/eunjin/Downloads/deleted_product_ids.csv


### 제거한 제품의 리뷰도 제거

In [19]:
import pandas as pd

# =========================
# 0) 파일 경로
# =========================
# 삭제된 product_id 목록 (앞에서 만든 파일)
deleted_ids_path = "/Users/eunjin/Downloads/deleted_product_ids.csv"

# 리뷰 데이터
reviews_path = "/Users/eunjin/Downloads/table1_reviews.csv"

# 출력 파일
out_reviews_path = "/Users/eunjin/Downloads/reviews_deduped.csv"

# =========================
# 1) 데이터 로드
# =========================
df_deleted_ids = pd.read_csv(deleted_ids_path)
df_reviews = pd.read_csv(reviews_path)

# 컬럼명
id_col = "product_id"

# 삭제 대상 product_id set
deleted_id_set = set(df_deleted_ids[id_col].astype(str))

print("=" * 80)
print("삭제 대상 product_id 개수:", len(deleted_id_set))
print("리뷰 원본 행 수:", df_reviews.shape[0])

# =========================
# 2) 리뷰 삭제 (product_id 기준)
# =========================
mask_delete = df_reviews[id_col].astype(str).isin(deleted_id_set)
df_reviews_deleted = df_reviews[mask_delete]
df_reviews_kept = df_reviews[~mask_delete]

# =========================
# 3) 결과 출력
# =========================
print("=" * 80)
print("[리뷰 삭제 결과]")
print("=" * 80)
print("삭제된 리뷰 수:", df_reviews_deleted.shape[0])
print("남은 리뷰 수:", df_reviews_kept.shape[0])

# (선택) 어떤 product_id 리뷰가 지워졌는지 요약
deleted_review_counts = (
    df_reviews_deleted.groupby(id_col)
    .size()
    .sort_values(ascending=False)
)

print("\n삭제된 리뷰가 있었던 product_id 상위 20개:")
print(deleted_review_counts.head(20).to_string())

# =========================
# 4) 저장
# =========================
df_reviews_kept.to_csv(out_reviews_path, index=False, encoding="utf-8-sig")
print(f"\n정리된 리뷰 파일 저장 완료: {out_reviews_path}")


삭제 대상 product_id 개수: 60
리뷰 원본 행 수: 16902
[리뷰 삭제 결과]
삭제된 리뷰 수: 3590
남은 리뷰 수: 13312

삭제된 리뷰가 있었던 product_id 상위 20개:
product_id
S104    100
S32     100
S248    100
S319    100
S316    100
S314    100
S329    100
S311    100
S177    100
S39     100
S109    100
S230    100
S163    100
S29     100
S15     100
S294    100
S50     100
S295    100
S296    100
S271     93

정리된 리뷰 파일 저장 완료: /Users/eunjin/Downloads/reviews_deduped.csv
